# 학습목표
keras 모델에서 과적합을 방지하기 위한 3가지 방법을 사용한다   
1. Weight Decay (가중치 감소)
2. Dropout (드롭아웃)
3. Early Stopping (조기 종료)



### Fashion MNIST 예제 사용 

In [10]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras import regularizers

import os
import numpy as np
import tensorflow as tf
import keras

In [11]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

In [12]:
X_train = X_train / 255.
X_test = X_test / 255.

베이스라인 모델 

In [13]:
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(64),
    Dense(10, activation='softmax')
])

### 1. Weight Decay (가중치 감소)

가중치가 너무 커지지 않도록 레이어에 가중치 규제항을 추가한다 

In [14]:
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(64,
          kernel_regularizer=regularizers.l2(0.01),   # L2 규제항
          activity_regularizer=regularizers.l1(0.01)),# L1 규제항 
    Dense(10, activation='softmax')
])

### 2. Dropout (드롭아웃) 
각 iteration 마다 레이어 노드 중 일부 비율을 사용하지 않도록 만듦  
(출력값을 0으로 설정하는 작동원리)

In [15]:
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(64,
          kernel_regularizer=regularizers.l2(0.01),   # L2 규제항
          activity_regularizer=regularizers.l1(0.01)),# L1 규제항
    Dropout(0.2),  # 20%의 노드에 드롭아웃 적용
    Dense(10, activation='softmax')
])

### 3. Early Stopping (조기 종료) 
과적합의 징후: 학습데이터에 대한 손실은 계속 줄어들지만, 검증 데이터에 대한 손실이 증가한다

위 과적합 징후가 나타났을 때 학습을 종료하도록 설정하는 방법  

In [16]:
# 조기종료 적용을 위한 설정들

# 파라미터 저장 경로
checkpoint_filepath = 'FMbest.hdf5'

# 조기종료 클래스 선언
early_stop = keras.callbacks.EarlyStopping(monitor='loss', # 조기종료 적용 기준
                                           min_delta=0,
                                           patience=10,        # 조기종료 조건
                                           verbose=1)

# 조기종료시 옵션을 저장하는 클래스 선언
save_best = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                               monitor='loss',
                                               verbose=1,
                                               save_best_only=True,
                                               save_weights_only=True,
                                               mode='auto',
                                               save_freq='epoch',
                                               options=None)

In [17]:
# 모델 컴파일
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1 = 0.89)
             , loss='sparse_categorical_crossentropy'
             , metrics=['accuracy'])

In [20]:
# callbacks 활성화하여 학습
model.fit(X_train, y_train,
          batch_size=32,
          epochs=30,
          verbose=1,
          callbacks=[early_stop, save_best])

Epoch 1/30
1872/1875 [============================>.] - ETA: 0s - loss: 0.7312 - accuracy: 0.8155
Epoch 1: loss did not improve from 0.72678
1875/1875 [==============================] - 5s 2ms/step - loss: 0.7311 - accuracy: 0.8155
Epoch 2/30
1867/1875 [============================>.] - ETA: 0s - loss: 0.7347 - accuracy: 0.8145
Epoch 2: loss did not improve from 0.72678
1875/1875 [==============================] - 5s 2ms/step - loss: 0.7350 - accuracy: 0.8145
Epoch 3/30
1865/1875 [============================>.] - ETA: 0s - loss: 0.7236 - accuracy: 0.8157
Epoch 3: loss improved from 0.72678 to 0.72377, saving model to FMbest.hdf5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.7238 - accuracy: 0.8156
Epoch 4/30
1857/1875 [============================>.] - ETA: 0s - loss: 0.7270 - accuracy: 0.8135
Epoch 4: loss did not improve from 0.72377
1875/1875 [==============================] - 5s 3ms/step - loss: 0.7270 - accuracy: 0.8136
Epoch 5/30
1867/1875 [=================

In [21]:
# best_parameter 모델 불러오기
model.load_weights(checkpoint_filepath)

test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=1)

313/313 [==============================] - 1s 2ms/step - loss: 0.7673 - accuracy: 0.8036
